In [4]:
import numpy as np
import sklearn.model_selection
from sklearn import datasets
import sklearn.metrics
import shutil
import autosklearn.classification
import autosklearn.regression
import pickle
import sys

import warnings
warnings.filterwarnings("ignore")

/home/jcui/.conda/envs/auto-sklearn/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/home/jcui/.conda/envs/auto-sklearn/lib/python3.7/site-packages/pyparsing.py:2910: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )


In [5]:
# Inputs: X, y, regression or classification
class base_multi_automl:
    def __init__(self, model_type):
        self.model_type = model_type
        pass
    
    # X and y are NP matrices, assumes groups is an np vector with indicators starting from 0
    def fit(self, X, y, groups)
        # Split X into a list of Xs based on group assignments
        
                        
        # Train each group on automl
                
        # Retrive a list of models bas ed on automl
        
        # Create a new dataset based on training each instance's groups for each model and supplying a prediction
        
        # Train this new dataset to get the weights for each model
        
        # Return a the list of models and the weights of each model
        
    def test(self, X, y, block_presence)
        # X and y should be in same structure as before
        # Block presence indicates which blocks are missing or not
        
        # Split X into the data groups
        # Train each group on the saved automl models/weights
        # Ensemble them with the group model normalized over present blocks        
        
        
        

SyntaxError: invalid syntax (<ipython-input-5-394322182f88>, line 8)

In [6]:
from pmlb import fetch_data

# Returns NumPy arrays
X, y = fetch_data('adult', return_X_y=True, local_cache_dir='./')
groups = np.array([0, 1, 2, 1, 1, 3, 1, 3, 0, 0, 1, 1, 1, 0])

In [7]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

In [8]:
m, d = np.shape(X)
total_group_num = np.size(np.unique(groups))

In [9]:
# Group Splitter

# Pass in X data and retrieve a list of blocks
def create_groups(X, groups):    
    total_group_num = np.size(np.unique(groups))    
    group_counts = [0] * total_group_num
    m, d = np.shape(X)

    # Create a list of the groups
    X_groups = [np.empty(1) for i in range(total_group_num)]
    for group_num in range(total_group_num):
        print('Group Num is {}.'.format(group_num))
        for feature in range(d):        
            if group_num == groups[feature]:
                group_counts[group_num] += 1
                if group_counts[group_num] == 1:
                    X_groups[group_num] = np.asmatrix(X[:, feature]).T                
                else:
                    X_groups[group_num] = np.hstack((X_groups[group_num], np.asmatrix(X[:, feature]).T))                 
    return X_groups, group_counts
                

X_groups, group_counts = create_groups(X_train, groups)

Group Num is 0.
Group Num is 1.
Group Num is 2.
Group Num is 3.


In [11]:
X_groups[2].shape

(36631, 1)

In [12]:
# Training Each Group on AutoML
time_cap = 30
per_limit = 15
block_models = [0] * total_group_num
for group_num in range(total_group_num):
    block_data = X_groups[group_num]
    automl_model = autosklearn.classification.AutoSklearnClassifier(
                time_left_for_this_task = time_cap,
                per_run_time_limit = per_limit)
    automl_model.fit(block_data, y_train)
    block_models[group_num] = pickle.dumps(automl_model)
    

[WARNING] [2019-04-14 15:46:23,286:EnsembleBuilder(1):16878d78c262aa5f260b41cff9e147de] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:46:23,298:EnsembleBuilder(1):16878d78c262aa5f260b41cff9e147de] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:46:25,303:EnsembleBuilder(1):16878d78c262aa5f260b41cff9e147de] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:46:47,860:EnsembleBuilder(1):584601c94f6dd94d45f599676e3bc094] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:46:47,872:EnsembleBuilder(1):584601c94f6dd94d45f599676e3bc094] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:46:49,876:EnsembleBuilder(1):584601c94f6dd94d45f599676e3bc094] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:47:12,504:EnsembleBuilder(1):54b2af14a5b0771e9f3cc58bd9d4286f] No models better than random - using Dummy Score!
[WARNING] [2019-04-14 15:47:12,516

In [31]:
# Create a data table for model predictions of each classifier
def getBlockPredictions(X_groups, block_models):
    m = X_groups[0].shape[0]
    total_group_num = len(X_groups)
    block_predictions = np.empty((m, total_group_num))
    for group_num in range(total_group_num):
        print('GROUP NUM ', group_num)
        group_automl_model = pickle.loads(block_models[group_num])

        block = X_groups[group_num]
        block_predictions[:, group_num] = group_automl_model.predict(block)
    return block_predictions
    
        

In [32]:
block_predictions = getBlockPredictions(X_groups, block_models)

GROUP NUM  0
GROUP NUM  1
GROUP NUM  2
GROUP NUM  3


In [26]:
# Create one final ensemble method to train on the block predictions for one final prediction
from sklearn.linear_model import LogisticRegression
final_model = LogisticRegression().fit(block_predictions, y_train)

# Evaluate training accuracy
final_model.score(block_predictions, y_train)



0.8691818405175944

In [28]:
# Convert testing X into block predictions array and evaluate testing accuracy
X_test_groups, group_counts = create_groups(X_test, groups)
block_predictions_test = getBlockPredictions(X_test_groups, block_models)
final_model.score(block_predictions_test, y_test)

Group Num is 0.
Group Num is 1.
Group Num is 2.
Group Num is 3.
GROUP NUM  0
GROUP NUM  1
GROUP NUM  2
GROUP NUM  3


0.8168864138891164

In [14]:
starts

[0, 4, 11, 12, 14]